In [1]:
import sys
import os
import yaml
os.chdir("/home/yw699/codes/LLM-Hallu")
sys.path.append(os.path.abspath("src"))
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"
from utils import *
setup_logger()

In [2]:
with open("configs/experiment_config1.yaml", "r") as file:
    config = yaml.safe_load(file)
    
wandb_config = config["wandb"]
metrics_config = config["metrics"]
experiment_details = {'config': config}

In [3]:
wandb_id = 'k0fk4l54'
if wandb_id == 'YOUR_ID':
    raise ValueError('Need to provide wandb_id of demo run!')

api = wandb.Api()
old_run = api.run(f'semantic_uncertainty/{wandb_id}')

print("Run Name:", old_run.name)
print("Run Config:", old_run.config)
print("Run Summary:", old_run.summary)


for file in old_run.files():
    print("File Name:", file.name)

Run Name: ethereal-smoke-21
Run Config: {'model': {'model_name': 'meta-llama/Llama-2-7b-hf', 'max_new_tokens': 50, 'stop_sequences': 'default'}, 'wandb': {'debug': False, 'project': 'semantic_uncertainty', 'experiment_lot': 'MyExperiment'}, 'p_true': {'p_true_hint': False, 'compute_p_true': True, 'get_training_set_generations': True, 'compute_accuracy_at_all_temps': True, 'get_training_set_generations_most_likely_only': True}, 'prompt': {'add_tag': True, 'few-shot': False, 'shot_num': 3, 'use_context': True, 'brief_always': True, 'prompt_template_path': '/home/yw699/codes/LLM-Hallu/data/prompt_templates/ask_templates/test2.txt'}, 'sample': {'temperature': 1, 'sample_count': 5, 'sampling_method': 'simple_sample'}, 'dataset': {'name': 'squad', 'seed': 42, 'answerable_only': True}, 'metrics': [{'name': 'p_true', 'p_true_num_fewshot': 2}, {'name': 'accuracy'}, {'name': 'diversity'}], 'p_true_num_fewshot': 2}
Run Summary: {'_runtime': 42.354001356, '_step': 0, '_timestamp': 1732048160.65792

In [4]:
user = os.environ['USER']
slurm_jobid = os.getenv('SLURM_JOB_ID', None)
scratch_dir = os.getenv('SCRATCH_DIR', '.')
entity = os.getenv('WANDB_SEM_UNC_ENTITY', None)
dir = f"{scratch_dir}/{user}/uncertainty"
if not os.path.exists(dir):
        os.makedirs(dir)
project = config["wandb"]["project"]
if config["wandb"]["debug"]:
    project = f"{project}_debug"
experiment_lot = config["wandb"]['experiment_lot']
notes=f'slurm_id: {slurm_jobid}, experiment_lot: {experiment_lot}'
wandb.init(
            entity=entity,
            project=project,
            dir=dir,
            notes=notes,
            config={**old_run.config},
        )

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yupuwang2001 (yupuwang2001-duke-university). Use `wandb login --relogin` to force relogin


In [5]:
def save_wandb(object, file):
    with open(f'{wandb.run.dir}/{file}', 'wb') as f:
        pickle.dump(object, f)
    wandb.save(f'{wandb.run.dir}/{file}')
    print(f"File saved and uploaded to WandB.")

In [6]:
save_dir = wandb.run.dir
for file in old_run.files():
    print(f"Download: {file.name}")
    
    file.download(root=save_dir, replace=True)
    
    file_name = file.name.split("/")[-1]
    
    original_path = os.path.join(save_dir, file.name)  
    new_path = os.path.join(save_dir, file_name) 
    
    os.rename(original_path, new_path)  
    print(f"File saved as: {new_path}")

    wandb.save(new_path)
for root, dirs, _ in os.walk(save_dir, topdown=False):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            if not os.listdir(dir_path):  
                os.rmdir(dir_path)
                print(f"Deleted empty directory: {dir_path}")

Download: requirements.txt
File saved as: ./yw699/uncertainty/wandb/run-20241119_203456-3rp0xrrf/files/requirements.txt
Download: wandb-metadata.json
File saved as: ./yw699/uncertainty/wandb/run-20241119_203456-3rp0xrrf/files/wandb-metadata.json
Download: yw699/uncertainty/wandb/run-20241119_152837-k0fk4l54/files/experiment_details.pkl
File saved as: ./yw699/uncertainty/wandb/run-20241119_203456-3rp0xrrf/files/experiment_details.pkl
Download: yw699/uncertainty/wandb/run-20241119_152837-k0fk4l54/files/train_generations.pkl
File saved as: ./yw699/uncertainty/wandb/run-20241119_203456-3rp0xrrf/files/train_generations.pkl
Download: yw699/uncertainty/wandb/run-20241119_152837-k0fk4l54/files/uncertainty_measures.pkl
File saved as: ./yw699/uncertainty/wandb/run-20241119_203456-3rp0xrrf/files/uncertainty_measures.pkl
Download: yw699/uncertainty/wandb/run-20241119_152837-k0fk4l54/files/validation_generations.pkl
File saved as: ./yw699/uncertainty/wandb/run-20241119_203456-3rp0xrrf/files/validat

In [7]:
import pickle

base_path = './yw699/uncertainty/wandb/run-20241119_154636-5b5le9ev/files'
file_name = "uncertainty_measures.pkl"

with open(f'{base_path}/{file_name}', 'rb') as f:
    result_dict = pickle.load(f)

print(result_dict)



file_name = "validation_generations.pkl"
with open(f'{base_path}/{file_name}', 'rb') as f:
    validation_generations = pickle.load(f)


print(validation_generations)


{'uncertainty_measures': {'p_false': [1.2268030792474747, 1.2279730141162872, 1.2496417760849], 'p_false_fixed': [0.20292227201728397, 0.2038542557579629, 0.22092018188747176]}}
{'5728eb1a3acd2414000e01c6': {'question': 'Why is the need for acceptance of punishment needed?', 'context': "Some civil disobedients feel it is incumbent upon them to accept punishment because of their belief in the validity of the social contract, which is held to bind all to obey the laws that a government meeting certain standards of legitimacy has established, or else suffer the penalties set out in the law. Other civil disobedients who favor the existence of government still don't believe in the legitimacy of their particular government, or don't believe in the legitimacy of a particular law it has enacted. And still other civil disobedients, being anarchists, don't believe in the legitimacy of any government, and therefore see no need to accept punishment for a violation of criminal law that does not inf

In [8]:
from collections import defaultdict
result_dict['semantic_ids'] = []
entropies = defaultdict(list)
validation_embeddings, validation_is_true, validation_answerable = [], [], []
p_trues = []
count = 0  

def is_answerable(generation):
    return len(generation['reference']['answers']['text']) > 0




In [9]:
use_all_generations = False
use_num_generations =2
num_eval_samples = 100

 ## compute_predictive_entropy

In [11]:
for idx, tid in enumerate(validation_generations):

        example = validation_generations[tid]
        question = example['question']
        context = example['context']
        full_responses = example["responses"]
        most_likely_answer = example['most_likely_answer']
        print(most_likely_answer)
        if not use_all_generations:
            if use_num_generations == -1:
                raise ValueError
            responses = [fr[0] for fr in full_responses[:use_num_generations]]
        else:
            responses = [fr[0] for fr in full_responses]

        validation_is_true.append(most_likely_answer['accuracy'])
        validation_answerable.append(is_answerable(example))
        #validation_embeddings.append(most_likely_answer['embedding'])
        logging.info('validation_is_true: %f', validation_is_true[-1])
        count += 1
        if count >= num_eval_samples:
            logging.info('Breaking out of main loop.')
            break
        if not use_all_generations:
            log_liks = [r[1] for r in full_responses[:use_num_generations]]
        else:
            log_liks = [r[1] for r in full_responses]
        for i in log_liks:
            assert i
        
        entropies['context_entails_response'].append(context_entails_response(context, responses, entailment_model))

        if args.condition_on_question and args.entailment_model == 'deberta':
                responses = [f'{question} {r}' for r in responses]
        semantic_ids = get_semantic_ids(responses, model=entailment_model,strict_entailment=args.strict_entailment, example=example)
        result_dict['semantic_ids'].append(semantic_ids)
        entropies['cluster_assignment_entropy'].append(cluster_assignment_entropy(semantic_ids))

        
        



2024-11-19 20:35:52 INFO     validation_is_true: 1.000000
2024-11-19 20:35:52 INFO     validation_is_true: 1.000000
2024-11-19 20:35:52 INFO     validation_is_true: 1.000000


{'response': 'social contract', 'token_log_likelihoods': [0.0, 0.0], 'accuracy': 1.0}
{'response': 'Duisburg', 'token_log_likelihoods': [0.0, 0.0, 0.0], 'accuracy': 1.0}
{'response': 'Lisbon Treaty', 'token_log_likelihoods': [0.0, 0.0, 0.0, 0.0], 'accuracy': 1.0}


In [ ]:
if args.compute_predictive_entropy:
            # Token log likelihoods. Shape = (n_sample, n_tokens)
            if not args.use_all_generations:
                log_liks = [r[1] for r in full_responses[:args.use_num_generations]]
            else:
                log_liks = [r[1] for r in full_responses]

            for i in log_liks:
                assert i

            if args.compute_context_entails_response:
                # Compute context entails answer baseline.
                entropies['context_entails_response'].append(context_entails_response(
                    context, responses, entailment_model))

            if args.condition_on_question and args.entailment_model == 'deberta':
                responses = [f'{question} {r}' for r in responses]

            # Compute semantic ids.
            semantic_ids = get_semantic_ids(
                responses, model=entailment_model,
                strict_entailment=args.strict_entailment, example=example)

            result_dict['semantic_ids'].append(semantic_ids)

            # Compute entropy from frequencies of cluster assignments.
            entropies['cluster_assignment_entropy'].append(cluster_assignment_entropy(semantic_ids))

            # Length normalization of generation probabilities.
            log_liks_agg = [np.mean(log_lik) for log_lik in log_liks]

            # Compute naive entropy.
            entropies['regular_entropy'].append(predictive_entropy(log_liks_agg))

            # Compute semantic entropy.
            log_likelihood_per_semantic_id = logsumexp_by_id(semantic_ids, log_liks_agg, agg='sum_normalized')
            pe = predictive_entropy_rao(log_likelihood_per_semantic_id)
            entropies['semantic_entropy'].append(pe)

            # pylint: disable=invalid-name
            log_str = 'semantic_ids: %s, avg_token_log_likelihoods: %s, entropies: %s'
            entropies_fmt = ', '.join([f'{i}:{j[-1]:.2f}' for i, j in entropies.items()])
            # pylint: enable=invalid-name
            logging.info(80*'#')
            logging.info('NEW ITEM %d at id=`%s`.', idx, tid)
            logging.info('Context:')
            logging.info(example['context'])
            logging.info('Question:')
            logging.info(question)
            logging.info('True Answers:')
            logging.info(example['reference'])
            logging.info('Low Temperature Generation:')
            logging.info(most_likely_answer['response'])
            logging.info('Low Temperature Generation Accuracy:')
            logging.info(most_likely_answer['accuracy'])
            logging.info('High Temp Generation:')
            logging.info([r[0] for r in full_responses])
            logging.info('High Temp Generation:')
            logging.info(log_str, semantic_ids, log_liks_agg, entropies_fmt)